# Proyecto (obtención dataset)

In [3]:
# --- General ---
import os
from datetime import datetime

# --- Data ---
import pandas as pd
from pandasql import sqldf
import numpy as np

# --- Conexión ---
import elasticsearch

# --- Procesamiento lenguaje: spacy ---
import spacy
# from spacy.matcher import Matcher
from spacy.matcher import PhraseMatcher


# ==============================================================================

# --- Configuración ---
# User password
password = os.environ.get('SOPHIA2')

# --- Funciones ---
# Cargar paquete de español mediano
nlp = spacy.load("es_core_news_md")

## Inicialización peticiones Sophia2

In [6]:
# --- Nombre archivo ---
nombre_archivo = 'data'
anotador = 'agarnica'

# --- Parameters queries ---
# País
country="chile"
# Fecha
from_="2022-04-01"
to_="2022-04-02"
nombre_archivo = nombre_archivo + '_' + from_ + '_' + to_ + '.csv'


# --- Data Connection ---
IP = "search.sophia2.org"
PORT = 9200
USER= "elastic"
PASS= password

# --- Connection ---
es = elasticsearch.Elasticsearch(
    IP,
    http_auth=(USER, PASS)
)

# --- Queries ---
query = { 
    "bool": { 
    "filter": [
        {"range": {
      "date": {
        "gte": from_,
        "lt": to_
      }
      }},

        { "term":  { "country": country }},
    ]
    }  
}

In [7]:
# --- Confirmación de conexión ---
res = es.search(index="news", query=query, size=10000)
print("Son %d noticias encontradas..." % res['hits']['total']['value'])

Son 190 noticias encontradas...


## Creación dataframe

In [8]:
# --- Columnas Dataframe ---
data = {'id_news':[],'country':[],'media_outlet':[],'url':[],'title':[],'text':[],'date':[]}
df = pd.DataFrame(data)  

# --- Información Dataframe---
for hit in res['hits']['hits']:

    id_news = hit['_source']['id_news']
    country = hit['_source']['country']
    media_outlet = hit['_source']['media_outlet']
    url = hit['_source']['url']
    title = hit['_source']['title']
    text = hit['_source']['text']
    date = hit['_source']['date']
    
    new_row = {'id_news':int(id_news), 'country':country, 'media_outlet':media_outlet, 'url':url, 'title':title, 'text':text, 'date':date}
    df = pd.concat([df, pd.DataFrame.from_records([new_row])])
    # df = df.append(new_row, ignore_index=True)


# Reseteamos el índice del dataframe para que sea secuencial, sino serían puros 0.
df = df.reset_index(drop=True)

# --- Elimina resiuduos ---
# Elimina duplicados
df = df.drop_duplicates(subset='url', keep='first')
# Elimina textos vacíos o con muy poco contenido
df = df[df['text'].str.len() > 50]

In [9]:
print(f"Son {len(df)} noticias encontradas que se utilizarán")
df.head()

Son 182 noticias encontradas que se utilizarán


,id_news,country,media_outlet,url,title,text,date
0,47135604.0,chile,tusnoticias,https://www.tusnoticias.cl/diputadosudi/,Diputados UDI piden información a municipalid...,"Los diputados de la UDI, Jorge Alessandri y Gu...",2022-04-01
2,47136121.0,chile,elaconcagua,https://www.elaconcagua.cl/2022/04/01/panquehu...,Panquehue implementa aplicación lectora de pat...,Con la firma de convenio entre la Municipalida...,2022-04-01
3,47136140.0,chile,clave9,https://www.clave9.cl/2022/04/01/dos-personas-...,Dos personas fallecidas dejó accidente de trán...,Dos personas fallecidas fue el saldo de un acc...,2022-04-01
4,47136168.0,chile,clave9,https://www.clave9.cl/2022/04/01/nuevo-ataque-...,Nuevo ataque incendiario dejó un camión destru...,Destruido por el fuego resultó un camión produ...,2022-04-01
6,47137055.0,chile,elobservador,https://www.observador.cl/pdi-detuvo-a-4-perso...,PDi detuvo a 4 personas por tráfico en Laguna ...,Se identificó dos domicilios en que se comerci...,2022-04-01


In [10]:
# Eliminamos country, media_outlet, url, text, date
df = df.drop(['country', 'media_outlet', 'url', 'text', 'date'], axis=1)


In [13]:
# Acortamos los datos a 30 y aniadimos una columna [anotador]
df = df.tail(20)
df['label'] = np.nan
df['anotador'] = anotador
df['label_ministerio'] = np.nan
df.head()

,id_news,title,label,anotador,label_ministerio
170,47687324.0,"""Especies invasoras"" en Los Ríos: un incómodo ...",NaN,agarnica,NaN
171,47687326.0,"Municipalidad renueva planes de sexualidad, af...",NaN,agarnica,NaN
172,47687334.0,Celebran 50 años de Radio Austral en Valdivia ...,NaN,agarnica,NaN
173,47687336.0,Desconocidos robaron en dependencias municipal...,NaN,agarnica,NaN
174,47687343.0,Casos activos de covid-19 siguen cayendo: este...,NaN,agarnica,NaN


## Guardamos la información CSV

In [14]:
# --- Info Archivo ---
print("El archivo se llama:")
print(nombre_archivo)

# --- Guarda archivo ---
df.to_csv("./data/"+nombre_archivo)

El archivo se llama:
data_2022-04-01_2022-04-02.csv


###  Con esto tenemos toda la información necesaria para cargar los datos y utilizarlos.